<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3 Interlude: Chisel Standard Library 
**Prev: 上一个小节：[Generators: Collections 生成器：集合](3.2_collections.ipynb)**<br>
**Next: 下一个小节：[Higher-Order Functions 高层次函数](3.3_higher-order_functions.ipynb)**

## Motivation 动机
Chisel is all about re-use, so it only makes sense to provide a standard library of interfaces (encouraging interoperability of RTL) and generators for commonly-used hardware blocks. Chisel 是为了复用而生，所以只有在提供一个标准接口库（激发与 RTL 的互通性）的时候它才有意义。

## Setup 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# The Cheatsheet 备忘清单
The [Chisel3 cheatsheet](https://chisel.eecs.berkeley.edu/doc/chisel-cheatsheet3.pdf) contains a summary of all the major hardware construction APIs, including some of the standard library utilities that we'll introduce below. [Chisel3 备忘清单](https://chisel.eecs.berkeley.edu/doc/chisel-cheatsheet3.pdf) 包含了一个所有主流硬件结构接口的总结，包括有一些标准库工具。我们将在本小节介绍。

# Decoupled: A Standard Ready-Valid Interface 去耦：一个标准 Ready-Valid 端口
One of the commonly used interfaces provided by Chisel is `DecoupledIO`, providing a ready-valid interface for transferring data. The idea is that the source drives the `bits` signal with the data to be transferred and the `valid` signal when there is data to be transferred. The sink drives the `ready` signal when it is ready to accept data, and data is considered transferred when both `ready` and `valid` are asserted on a cycle. Chisel 提供的有一个经常使用的端口是 `DecoupledIO`，它为传输数据提供了一个 ready-valid 端口。原理是发送端通过 `bits` 信号发送将要传递的数据，并且在有数据需要传输的时候发送 `valid` 信号。接收端在准备好接受数据时，发送 `ready` 信号。只有 `ready` 和 `valid` 在同一个时钟周期内都是高电平，数据才会被传递。

This provides a flow control mechanism in both directions for data transfer, including a backpressure mechanism. 这会为数据传输双方将提供一个流程控制机制，包括背压机制。

Note: `ready` and `valid` should not be combinationally coupled, otherwise this may result in unsynthesizable combinational loops. `ready` should only be dependent on whether the sink is able to receive data, and `valid` should only be dependent on whether the source has data. Only after the transaction (on the next clock cycle) should the values update. 注意：`ready` 和 `valid` 不应该耦合起来，不然这会导致异步组合循环。`ready` 只应该依赖于是否接收端能够接受数据，并且 `valid` 应该只依赖于是否发送端有数据需要发送。只有在当前事务之后（下一个时钟周期）两个信号的电平才可以更新。

Any Chisel data can be wrapped in a `DecoupledIO` (used as the `bits` field) as follows: 任何 Chisel 数据都可以被 `DecoupledIO` （被当作 `bits` 字段）包裹着，就像下面这样：

```scala
val myChiselData = UInt(8.W)
// or any Chisel data type, such as Bool(), SInt(...), or even custom Bundles 或者其他任何的 Chisel 数据类型，比如 Bool(), SInt(...), 甚至是自定义的 Bundles。
val myDecoupled = Decoupled(myChiselData)
```

The above creates a new `DecoupledIO` Bundle with fields 上面的代码声明了一个新的带有字段的 `DecoupledIO` Bundle。
- `valid`: Output(Bool)
- `ready`: Input(Bool)
- `bits`: Output(UInt(8.W))
___

The rest of the section will be structured somewhat differently from the ones before: instead of giving you coding exercises, we're going to give some code examples and testcases that print the circuit state. Try to predict what will be printed before just running the tests. 本小节的剩余部分将有一些不同于之前的东西：我们将会给出一些代码样例和输出电路状态的测试样例而不是给你编程练习。尝试着在测试运行之前预测打印输出的结果。

## Queues 队列

`Queue` creates a FIFO (first-in, first-out) queue with Decoupled interfaces on both sides, allowing backpressure. Both the data type and number of elements are configurable.  `Queue` 对应硬件的 FIFO，两端都带有去耦合端口的队列运行背压机制。数据类型和元素的数量都是可配置的。

In [ ]:
Driver(() => new Module {
    // Example circuit using a Queue 使用队列的电路样例
    val io = IO(new Bundle {
      val in = Flipped(Decoupled(UInt(8.W)))
      val out = Decoupled(UInt(8.W))
    })
    val queue = Queue(io.in, 2)  // 2-element queue 含有两个元素的队列
    io.out <> queue
  }) { c => new PeekPokeTester(c) {
    // Example testsequence showing the use and behavior of Queue 展示队列的使用方式和行为的测试样例
    poke(c.io.out.ready, 0)
    poke(c.io.in.valid, 1)  // Enqueue an element 让一个元素入队
    poke(c.io.in.bits, 42)
    println(s"Starting:")
    println(s"\tio.in: ready=${peek(c.io.in.ready)}")
    println(s"\tio.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    poke(c.io.in.valid, 1)  // Enqueue another element 让另外一个元素入队
    poke(c.io.in.bits, 43)
    // What do you think io.out.valid and io.out.bits will be? 你认为 io.out.valid 和 io.out.bits 的值将会是多少？
    println(s"After first enqueue: 第一次入队之后：")
    println(s"\tio.in: ready=${peek(c.io.in.ready)}")
    println(s"\tio.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    poke(c.io.in.valid, 1)  // Read a element, attempt to enqueue 读一个元素，尝试入队。
    poke(c.io.in.bits, 44)
    poke(c.io.out.ready, 1)
    // What do you think io.in.ready will be, and will this enqueue succeed, and what will be read? 你认为 io.in.ready 的值将会是多少？入队会成功吗？什么会被读到？
    println(s"On first read: 第一次读")
    println(s"\tio.in: ready=${peek(c.io.in.ready)}")
    println(s"\tio.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    poke(c.io.in.valid, 0)  // Read elements out 将元素读出
    poke(c.io.out.ready, 1)
    // What do you think will be read here? 你认为在这里会被读到什么值？
    println(s"On second read: 第二次读：")
    println(s"\tio.in: ready=${peek(c.io.in.ready)}")
    println(s"\tio.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
  
    // Will a third read produce anything? 第三次读会产生什么值吗？
    println(s"On third read:")
    println(s"\tio.in: ready=${peek(c.io.in.ready)}")
    println(s"\tio.out: valid=${peek(c.io.out.valid)}, bits=${peek(c.io.out.bits)}")
    step(1)
} }

## Arbiters 仲裁器
Arbiters routes data from _n_ `DecoupledIO` sources to one `DecoupledIO` sink, given a prioritization. 仲裁器使得数据从 _n_ `DecoupledIO` 发送端向一个 `DecoupledIO` 接收端传送，并且数据传送时带有优先级。
There are two types included in Chisel: 在 Chisel 中有两种类型：
- `Arbiter`: prioritizes lower-index producers   `Arbiter`：按照发送器降序形成优先次序。
- `RRArbiter`: runs in round-robin order   `RRArbiter`：循环顺序运行

Note that Arbiter routing is implemented in combinational logic. 注意：仲裁器是使用组合逻辑实现。

The below example will demonstrate the use of the priority arbiter (which you will also implement in the next section): 下面的例子将会展示优先级仲裁器的使用（在下一个小节中也会实现它）

In [ ]:
Driver(() => new Module {
    // Example circuit using a priority arbiter 使用优先级仲裁器的电路样例
    val io = IO(new Bundle {
      val in = Flipped(Vec(2, Decoupled(UInt(8.W))))
      val out = Decoupled(UInt(8.W))
    })
    // Arbiter doesn't have a convenience constructor, so it's built like any Module 仲裁器没有一个方便的构造模块，所以它像其他模块一样构造。
    val arbiter = Module(new Arbiter(UInt(8.W), 2))  // 2 to 1 Priority Arbiter 二选一优先级仲裁器。
    arbiter.io.in <> io.in
    io.out <> arbiter.io.out
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in(0).valid, 0)
    poke(c.io.in(1).valid, 0)
    println(s"Start:")
    println(s"\tin(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}")
    println(s"\tout.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
    poke(c.io.in(1).valid, 1)  // Valid input 1  valid 信号输入 1
    poke(c.io.in(1).bits, 42)
    // What do you think the output will be? 你认为输出将会是什么呢？
    println(s"valid input 1:    valid 信号输入 1：")
    println(s"\tin(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}")
    println(s"\tout.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
    poke(c.io.in(0).valid, 1)  // Valid inputs 0 and 1   valid 信号输入 0 和 1
    poke(c.io.in(0).bits, 43)
    // What do you think the output will be? Which inputs will be ready? 你认为输出会是什么呢？哪一个输入将会准备好？
    println(s"valid inputs 0 and 1:   valid 信号输入 0 和 1")
    println(s"\tin(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}")
    println(s"\tout.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
    poke(c.io.in(1).valid, 0)  // Valid input 0   valid 信号输入 0
    // What do you think the output will be? 你认为输出信号将会是什么呢？
    println(s"valid input 0:      valid 信号输入 0：")
    println(s"\tin(0).ready=${peek(c.io.in(0).ready)}, in(1).ready=${peek(c.io.in(1).ready)}")
    println(s"\tout.valid=${peek(c.io.out.valid)}, out.bits=${peek(c.io.out.bits)}")
} }

# Misc Function Blocks 混杂的函数块
Chisel Utils has some helpers that perform stateless functions. Chisel 程序有一些无边界函数

## Bitwise Utilities 按位的函数
### PopCount
PopCount returns the number of high (1) bits in the input as a `UInt`. `PopCount` 返回输入信号中，高电平的数量，类型是 `UInt`。

### Reverse 反转
Reverse returns the bit-reversed input. `Reverse` 返回输入的按位反转结果。

In [ ]:
Driver(() => new Module {
    // Example circuit using Reverse 使用反转的电路样例（应该是写错了，正确为   使用 PopCount 的电路样例）
    val io = IO(new Bundle {
      val in = Input(UInt(8.W))
      val out = Output(UInt(8.W))
    })
    io.out := PopCount(io.in)
  }) { c => new PeekPokeTester(c) {
    // Integer.parseInt is used create an Integer from a binary specification  使用 Integer.parseInt，使一个二进制字符转变为二进制整数
    poke(c.io.in, Integer.parseInt("00000000", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    poke(c.io.in, Integer.parseInt("00001111", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    poke(c.io.in, Integer.parseInt("11001010", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    poke(c.io.in, Integer.parseInt("11111111", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
} }

In [ ]:
Driver(() => new Module {
    // Example circuit using Reverse 使用反转的电路样例
    val io = IO(new Bundle {
      val in = Input(UInt(8.W))
      val out = Output(UInt(8.W))
    })
    io.out := Reverse(io.in)
  }) { c => new PeekPokeTester(c) {
    // Integer.parseInt is used create an Integer from a binary specification   使用 Integer.parseInt，使一个二进制字符转变为二进制整数
    poke(c.io.in, Integer.parseInt("01010101", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=0b${peek(c.io.out).toInt.toBinaryString}")
  
    poke(c.io.in, Integer.parseInt("00001111", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=0b${peek(c.io.out).toInt.toBinaryString}")
  
    poke(c.io.in, Integer.parseInt("11110000", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=0b${peek(c.io.out).toInt.toBinaryString}")
  
    poke(c.io.in, Integer.parseInt("11001010", 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=0b${peek(c.io.out).toInt.toBinaryString}")
} }

## OneHot encoding utilities 独热码的函数
OneHot is an encoding of integers where there is one wire for each value, and exactly one wire is high. This allows the efficient creation of some functions, for example muxes. However, behavior may be undefined if the one-wire-high condition is not held. 独热码是一个整数型编码，每一次只有一位是高电平。这产生了一些高效的函数，比如说多路选择器。然而，如果一位高位的状态没有保持的话，那么对应的行为可能是没有被定义的。

The below two functions provide conversion between binary (`UInt`) and OneHot encodings, and are inverses of each other: 下面两个函数提供了从二进制无符号整数 (`UInt`) 到独热码的转换，两个是相反的。
- UInt to OneHot: `UIntToOH`
- OneHot to UInt: `OHToUInt`

In [ ]:
Driver(() => new Module {
    // Example circuit using UIntToOH 使用无符号整数到独热码的电路样例
    val io = IO(new Bundle {
      val in = Input(UInt(4.W))
      val out = Output(UInt(16.W))
    })
    io.out := UIntToOH(io.in)
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in, 0)
    println(s"in=${peek(c.io.in)}, out=0b${peek(c.io.out).toInt.toBinaryString}")

    poke(c.io.in, 1)
    println(s"in=${peek(c.io.in)}, out=0b${peek(c.io.out).toInt.toBinaryString}")
  
    poke(c.io.in, 8)
    println(s"in=${peek(c.io.in)}, out=0b${peek(c.io.out).toInt.toBinaryString}")
  
    poke(c.io.in, 15)
    println(s"in=${peek(c.io.in)}, out=0b${peek(c.io.out).toInt.toBinaryString}")
} }

In [ ]:
Driver(() => new Module {
    // Example circuit using OHToUInt 使用独热码到无符号整数的电路样例
    val io = IO(new Bundle {
      val in = Input(UInt(16.W))
      val out = Output(UInt(4.W))
    })
    io.out := OHToUInt(io.in)
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in, Integer.parseInt("0000 0000 0000 0001".replace(" ", ""), 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")

    poke(c.io.in, Integer.parseInt("0000 0000 1000 0000".replace(" ", ""), 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    poke(c.io.in, Integer.parseInt("1000 0000 0000 0000".replace(" ", ""), 2)) //这里原来是 1000 0000 0000 0001
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    // Some invalid inputs: 一些无效的输入
    // None high 没有高电平
    poke(c.io.in, Integer.parseInt("0000 0000 0000 0000".replace(" ", ""), 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
  
    // Multiple high 多个高电平
    poke(c.io.in, Integer.parseInt("0001 0100 0010 0000".replace(" ", ""), 2))
    println(s"in=0b${peek(c.io.in).toInt.toBinaryString}, out=${peek(c.io.out)}")
} }

## Muxes 多路选择器
These muxes take in a list of values with select signals, and output the value associated with the lowest-index select signal. 这些多路选择器接受一列信号，其中包括选择信号。输出值与最低位索引选择信号有关

These can either take a list of (select: Bool, value: Data) tuples, or corresponding lists of selects and values as arguments. For simplicity, the examples below only demonstrate the second form. 这些可能会需要多个元组（比如选择用 Bool，值用 Data），或者对应的选择和值的列表作为参数。为了简单起见，下面的例子只展示了第二种方式。

### Priority Mux 优先多路选择器
A `PriorityMux` outputs the value associated with the lowest-index asserted select signal. 一个 `PriorityMux` 输出值与最低索引选择信号有关。

### OneHot Mux 独热码多路选择器
An `Mux1H` provides an efficient implementation when it is guaranteed that exactly one of the select signals will be high. Behavior is undefined if the assumption is not true. 当确定只有一个信号是高电平时，`Mux1H` 会提供一个有效的实现方式。如果假设不成立，那么实际效果是未知的。

In [ ]:
Driver(() => new Module {
    // Example circuit using PriorityMux 使用优先多路选择器的电路样例
    val io = IO(new Bundle {
      val in_sels = Input(Vec(2, Bool()))
      val in_bits = Input(Vec(2, UInt(8.W)))
      val out = Output(UInt(8.W))
    })
    io.out := PriorityMux(io.in_sels, io.in_bits)
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in_bits(0), 10)
    poke(c.io.in_bits(1), 20)
  
    // Select higher index only 只选择高位
    poke(c.io.in_sels(0), 0)
    poke(c.io.in_sels(1), 1)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
  
    // Select both - arbitration needed 两个都选择——需要仲裁器
    poke(c.io.in_sels(0), 1)
    poke(c.io.in_sels(1), 1)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
  
    // Select lower index only 只选择低位
    poke(c.io.in_sels(0), 1)
    poke(c.io.in_sels(1), 0)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
} }

In [ ]:
Driver(() => new Module {
    // Example circuit using Mux1H 使用 Mux1H 的电路样例
    val io = IO(new Bundle {
      val in_sels = Input(Vec(2, Bool()))
      val in_bits = Input(Vec(2, UInt(8.W)))
      val out = Output(UInt(8.W))
    })
    io.out := Mux1H(io.in_sels, io.in_bits)
  }) { c => new PeekPokeTester(c) {
    poke(c.io.in_bits(0), 10)
    poke(c.io.in_bits(1), 20)
  
    // Select index 1 选择高位
    poke(c.io.in_sels(0), 0)
    poke(c.io.in_sels(1), 1)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
  
    // Select index 0 选择低位
    poke(c.io.in_sels(0), 1)
    poke(c.io.in_sels(1), 0)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
  
    // Select none (invalid) 一个都不选择（无效）
    poke(c.io.in_sels(0), 0)
    poke(c.io.in_sels(1), 0)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
  
    // Select both (invalid) 两个都选择（无效）
    poke(c.io.in_sels(0), 1)
    poke(c.io.in_sels(1), 1)
    println(s"in_sels=${peek(c.io.in_sels)}, out=${peek(c.io.out)}")
} }

## Counter 计数器
`Counter` is a counter that can be incremented once every cycle, up to some specified limit, at which point it overflows. Note that it is **not** a Module, and its value is accessible. `Counter`是计数器，它可以在每一个周期自增，直到某个特定的限制，在那时它将会溢出。注意，计数器**不是**一个模块，并且它的值是可以读取的。

In [ ]:
Driver(() => new Module {
    // Example circuit using Mux1H 使用独热码选择器的电路样例
    val io = IO(new Bundle {
      val count = Input(Bool())
      val out = Output(UInt(2.W))
    })
    val counter = Counter(3)  // 3-count Counter (outputs range [0...2])  数三次的计数器（输出范围是 0-2）
    when(io.count) {
      counter.inc()
    }
    io.out := counter.value
  }) { c => new PeekPokeTester(c) {
    poke(c.io.count, 1)
    println(s"start: counter value=${peek(c.io.out)}")
  
    step(1)
    println(s"step 1: counter value=${peek(c.io.out)}")
  
    step(1)
    println(s"step 2: counter value=${peek(c.io.out)}")
  
    poke(c.io.count, 0)
    step(1)
    println(s"step without increment: counter value=${peek(c.io.out)}")
  
    poke(c.io.count, 1)
    step(1)
    println(s"step again: counter value=${peek(c.io.out)}")
} }

---
# You're done! 恭喜你，完成了本节内容的学习！

[Return to the top. 回到顶层。](#top)